In [5]:
import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import matplotlib.pyplot as plt
import time

print("Imports Complete")

Imports Complete


In [19]:
# Path to dataset: structured as "dataset/A/A1.jpg", "dataset/B/B2.jpg", etc.
DATASET_PATH = r"C:\Users\Bex\OneDrive\Documents\GitHub\asl_dataset\asl_dataset"  #Replace with your dataset path
IMAGE_SIZE = (100, 100)

def load_images_and_labels(dataset_path):
    X = []
    y = []
    for label in sorted(os.listdir(dataset_path)):
        label_path = os.path.join(dataset_path, label)
        if not os.path.isdir(label_path):
            continue
        for img_name in os.listdir(label_path):
            img_path = os.path.join(label_path, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                continue
            img = cv2.resize(img, IMAGE_SIZE)
            X.append(img)
            y.append(label)
    return np.array(X), np.array(y)

In [27]:
#Reload Data
X, y = load_images_and_labels(DATASET_PATH)

# Example: reshape and normalize data (assuming 28x28 grayscale images)
# Update these shapes according to your actual data
X = X.reshape(-1, 100, 100, 1).astype("float32")/255.0  # Normalize and reshape

from collections import Counter

# print("Label distribution:", Counter(y))

# Encode labels 
le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_categorical = to_categorical(y_encoded)

"""
print("y_encoded shape:", y_encoded.shape)
print("y_categorical shape:", y_categorical.shape)

print("X shape:", X.shape)
print("First label:", y[0])
print("Encoded labels:", y_encoded[:5])
"""

# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, shuffle=True, stratify=y_encoded)

# Define CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(y_categorical.shape[1], activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train CNN with timing
print("Training classifier...")
start_train = time.time()
model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=1)
end_train = time.time()
print(f"Training completed in {end_train - start_train:.4f} seconds.")

# Evaluate CNN with timing
print("Evaluating...")
start_eval = time.time()
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test, axis=1)
end_eval = time.time()
print(f"Evaluation completed in {end_eval - start_eval:.4f} seconds.")
print(classification_report(y_true, y_pred))

Training classifier...
Epoch 1/20


C:\Users\Bex\anaconda3\envs\tf-env\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


46/46 ━━━━━━━━━━━━━━━━━━━━ 3s 53ms/step - accuracy: 0.2622 - loss: 2.5937
Epoch 2/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 53ms/step - accuracy: 0.8464 - loss: 0.4897
Epoch 3/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 53ms/step - accuracy: 0.9086 - loss: 0.2672
Epoch 4/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 53ms/step - accuracy: 0.9498 - loss: 0.1553
Epoch 5/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 53ms/step - accuracy: 0.9551 - loss: 0.1135
Epoch 6/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 53ms/step - accuracy: 0.9617 - loss: 0.0937
Epoch 7/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 53ms/step - accuracy: 0.9743 - loss: 0.0833
Epoch 8/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 53ms/step - accuracy: 0.9907 - loss: 0.0373
Epoch 9/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 53ms/step - accuracy: 0.9798 - loss: 0.0512
Epoch 10/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 53ms/step - accuracy: 0.9809 - loss: 0.0514
Epoch 11/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 53ms/step - accuracy: 0.9900 - loss: 0.0316
Epoch 12/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 53ms/step - accuracy: 0.9967 - l

In [23]:
test_loss, test_accuracy = model.evaluate(X_test,y_test, verbose = 1)
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.9846 - loss: 0.0854
Test Accuracy: 0.9835
Test Loss: 0.0692
